In [34]:
import pandas as pd
from sklearn.metrics import mutual_info_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [3]:
link = 'https://raw.githubusercontent.com/farisknight13/Python_ML/main/%E0%B8%99%E0%B9%89%E0%B8%B3%E0%B8%A1%E0%B8%B1%E0%B8%99%E0%B8%A3%E0%B8%A7%E0%B8%A1.csv'
df = pd.read_csv(link)
df

,CALENDAR_YEAR,MONTH_DESC,MONTH_CD,TAX_AMT,TOTAL_TAX_AMT,VOLUMN,TOTAL_VOLUMN,PRICE,PRICE_FACTORY,TOTAL_VOLUMN_CAPA,CAPA,VAL_RATE,TAXQTY_UNIT,ART_TAXV,ART_TOTALTAXV,LAG_TIME_TOTAL_TAX_AMT,LAG_TIME_TAX_AMT,LAG_TIME_TOTAL_VOLUMN,LAG_TIME_VOLUMN,TOTAL_GASOLINE,KEROSENE,TOTAL_DIESEL,LPG,TOTAL_Fuel,GDP_Agriculture,GDP_Non_Agriculture,Total_GDP,Pop_up_15,Total_Labor_Froce,Labor_Agr,Labor_Non_Agr,P_ULG95,P_HSD,P_LPG,Crude_WPrice,Change_OP,CPI,Inf,Inter,Exr,U_Em,Total_Auto,Total_Auto_Petrol,Total_Auto_Diesel,Total_Auto_LPG,Total_Auto_CNG,Total_Auto_Electricity,Total_Auto_Hybrid,CPI_OA,W_Avr,W_Pri,W_Plu
0,2554,มกราคม,1,2.222302e+10,1.371691e+10,4.344679e+09,2.863381e+09,0.000000,0.000000,3.633069e+07,1.286312e+07,11.053448,5.076811,3.704450,5.150043,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,633.632,1.172,1695.860,1105.3944,4113.0854,164351.67,1871498.67,2047414.0,54279.62,38619.17,13389.40,24537.52,25.030339,26.414387,10.43150,110.686000,2.815500,99.65,3.37,5.32,31.5755,0.81,30322761.0,21592247.0,7405637.0,843450.0,265920,6312.0,22767.0,106.330000,10547.01,8964.45,16943.19
1,2554,กุมภาพันธ์,2,2.164869e+10,1.407221e+10,4.220640e+09,2.914440e+09,0.000000,0.000000,7.478745e+07,3.279386e+06,11.229805,4.969179,5.021377,6.429500,1.371691e+10,2.222302e+10,2.863381e+09,4.344679e+09,605.778,1.391,1698.912,1013.9074,3948.4414,164701.33,1937753.33,2109519.0,54322.32,38798.15,13449.78,24607.89,26.664445,27.770914,10.91669,119.327000,8.641000,101.33,3.34,5.25,30.7303,0.66,30483912.0,21695114.0,7444115.0,856446.0,270762,6228.0,23945.0,108.550000,10579.26,8778.73,17839.65
2,2554,มีนาคม,3,2.511702e+10,1.353646e+10,4.896327e+09,2.865928e+09,0.000000,0.000000,3.713671e+07,3.346770e+07,11.589345,5.101931,3.734156,5.349773,1.407221e+10,2.164869e+10,2.914440e+09,4.220640e+09,639.865,1.342,1831.691,1152.6093,4269.9543,165051.00,2004008.00,2171624.0,54365.40,38955.14,13506.88,24688.20,27.047719,27.639726,10.27190,125.445455,6.118455,98.57,3.45,5.25,30.6901,0.73,30667049.0,21809442.0,7490151.0,872606.0,275828,6133.0,25009.0,110.260000,10817.51,9267.39,17432.88
3,2554,เมษายน,4,2.261955e+10,1.324597e+10,4.597110e+09,2.848384e+09,0.000000,0.000000,3.694402e+07,3.134440e+07,11.864044,4.974018,3.488816,5.101580,1.353646e+10,2.511702e+10,2.865928e+09,4.896327e+09,614.701,1.244,1724.461,1123.4093,4106.2233,163452.00,2024071.00,2185274.0,54408.85,38909.27,13572.62,24571.16,27.184130,27.380147,10.25900,119.421667,-6.023788,98.49,2.47,5.24,30.8884,0.97,30804836.0,21887534.0,7531226.0,886244.0,279621,6059.0,25598.0,111.583333,11453.95,9750.22,18382.53
4,2554,พฤษภาคม,5,8.343328e+09,4.914525e+09,4.743274e+09,2.967009e+09,0.000000,0.000000,6.134105e+07,5.331400e+07,11.384126,3.887956,2.687062,3.752981,1.324597e+10,2.261955e+10,2.848384e+09,4.597110e+09,632.200,1.261,1806.773,1154.9130,4307.4840,161853.00,2044134.00,2198924.0,54452.70,39006.98,14393.90,23874.57,25.156665,25.870855,10.32540,110.339524,-9.082143,99.20,2.52,5.23,31.3373,0.92,31037272.0,22033426.0,7590982.0,905418.0,284986,5963.0,26932.0,109.716667,11137.12,9548.08,17200.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2562,สิงหาคม,8,3.106545e+10,2.093361e+10,5.434510e+09,3.818035e+09,49.936558,67.880428,5.201406e+09,2.133556e+08,0.009600,5.100187,10.758439,11.020983,2.248958e+10,3.238666e+10,4.066556e+09,5.636718e+09,0.000,0.000,0.000,0.0000,0.0000,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00000,44.736000,1.493826,0.00,0.00,0.00,31.2150,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.00,0.00,0.00
104,2562,กันยายน,9,3.081077e+10,2.113849e+10,5.380393e+09,3.795921e+09,471.270093,490.512497,4.631266e+09,1.352323e+08,0.064000,5.085954,7.666068,7.959199,2.093361e+10,3.106545e+10,3.818035e+09,5.434510e+09,0.000,0.000,0.000,0.0000,0.0000,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.00,0.00,0.0000

# **Non-normalization**

## แบ่งข้อมูลTrain 2554-2561 Test 2562

In [4]:
feature_train = df.iloc[:96,:19]
label_train = df.iloc[:96,4]
feature_test = df.iloc[96:,:19]
label_test = df.iloc[96:,4]
feature_train

,CALENDAR_YEAR,MONTH_DESC,MONTH_CD,TAX_AMT,TOTAL_TAX_AMT,VOLUMN,TOTAL_VOLUMN,PRICE,PRICE_FACTORY,TOTAL_VOLUMN_CAPA,CAPA,VAL_RATE,TAXQTY_UNIT,ART_TAXV,ART_TOTALTAXV,LAG_TIME_TOTAL_TAX_AMT,LAG_TIME_TAX_AMT,LAG_TIME_TOTAL_VOLUMN,LAG_TIME_VOLUMN
0,2554,มกราคม,1,2.222302e+10,1.371691e+10,4.344679e+09,2.863381e+09,0.000000,0.000000,3.633069e+07,1.286312e+07,11.053448,5.076811,3.704450,5.150043,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,2554,กุมภาพันธ์,2,2.164869e+10,1.407221e+10,4.220640e+09,2.914440e+09,0.000000,0.000000,7.478745e+07,3.279386e+06,11.229805,4.969179,5.021377,6.429500,1.371691e+10,2.222302e+10,2.863381e+09,4.344679e+09
2,2554,มีนาคม,3,2.511702e+10,1.353646e+10,4.896327e+09,2.865928e+09,0.000000,0.000000,3.713671e+07,3.346770e+07,11.589345,5.101931,3.734156,5.349773,1.407221e+10,2.164869e+10,2.914440e+09,4.220640e+09
3,2554,เมษายน,4,2.261955e+10,1.324597e+10,4.597110e+09,2.848384e+09,0.000000,0.000000,3.694402e+07,3.134440e+07,11.864044,4.974018,3.488816,5.101580,1.353646e+10,2.511702e+10,2.865928e+09,4.896327e+09
4,2554,พฤษภาคม,5,8.343328e+09,4.914525e+09,4.743274e+09,2.967009e+09,0.000000,0.000000,6.134105e+07,5.331400e+07,11.384126,3.887956,2.687062,3.752981,1.324597e+10,2.261955e+10,2.848384e+09,4.597110e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2561,สิงหาคม,8,3.297169e+10,1.971553e+10,5.768247e+09,3.627353e+09,459.402081,8.583704,3.774096e+09,9.677267e+07,0.234029,4.972036,10.560527,10.864531,1.931377e+10,3.168121e+10,3.567040e+09,5.630687e+09
92,2561,กันยายน,9,4.217391e+10,2.926961e+10,7.367896e+09,5.314754e+09,420.296695,59.373953,5.553845e+09,5.147209e+07,0.055230,5.021154,11.784984,12.151931,1.971553e+10,3.297169e+10,3.627353e+09,5.768247e+09
93,2561,ตุลาคม,10,1.354485e+10,5.803831e+09,2.403801e+09,1.125814e+09,538.496453,68.474851,1.364925e+09,5.817272e+07,0.213278,4.982616,11.238868,11.470734,2.926961e+10,4.217391e+10,5.314754e+09,7.367896e+09
94,2561,พฤศจิกายน,11,3.054517e+10,2.168809e+10,5.330737e+09,3.912047e+09,613.466892,573.311932,4.018208e+09,1.227488e+08,0.250349,5.047734,11.525254,11.754408,5.803831e+09,1.354485e+10,1.125814e+09,2.403801e+09


##Drop คอลัมที่ไม่ได้เกี่ยวกับการคำนวณ เช่น ปี เดือน คำตอบ

In [5]:
feature_train.drop(['CALENDAR_YEAR', 'MONTH_CD','MONTH_DESC','TOTAL_TAX_AMT'], axis=1, inplace=True)
feature_test.drop(['CALENDAR_YEAR', 'MONTH_CD','MONTH_DESC','TOTAL_TAX_AMT'], axis=1, inplace=True)
feature_train


,TAX_AMT,VOLUMN,TOTAL_VOLUMN,PRICE,PRICE_FACTORY,TOTAL_VOLUMN_CAPA,CAPA,VAL_RATE,TAXQTY_UNIT,ART_TAXV,ART_TOTALTAXV,LAG_TIME_TOTAL_TAX_AMT,LAG_TIME_TAX_AMT,LAG_TIME_TOTAL_VOLUMN,LAG_TIME_VOLUMN
0,2.222302e+10,4.344679e+09,2.863381e+09,0.000000,0.000000,3.633069e+07,1.286312e+07,11.053448,5.076811,3.704450,5.150043,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,2.164869e+10,4.220640e+09,2.914440e+09,0.000000,0.000000,7.478745e+07,3.279386e+06,11.229805,4.969179,5.021377,6.429500,1.371691e+10,2.222302e+10,2.863381e+09,4.344679e+09
2,2.511702e+10,4.896327e+09,2.865928e+09,0.000000,0.000000,3.713671e+07,3.346770e+07,11.589345,5.101931,3.734156,5.349773,1.407221e+10,2.164869e+10,2.914440e+09,4.220640e+09
3,2.261955e+10,4.597110e+09,2.848384e+09,0.000000,0.000000,3.694402e+07,3.134440e+07,11.864044,4.974018,3.488816,5.101580,1.353646e+10,2.511702e+10,2.865928e+09,4.896327e+09
4,8.343328e+09,4.743274e+09,2.967009e+09,0.000000,0.000000,6.134105e+07,5.331400e+07,11.384126,3.887956,2.687062,3.752981,1.324597e+10,2.261955e+10,2.848384e+09,4.597110e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,3.297169e+10,5.768247e+09,3.627353e+09,459.402081,8.583704,3.774096e+09,9.677267e+07,0.234029,4.972036,10.560527,10.864531,1.931377e+10,3.168121e+10,3.567040e+09,5.630687e+09
92,4.217391e+10,7.367896e+09,5.314754e+09,420.296695,59.373953,5.553845e+09,5.147209e+07,0.055230,5.021154,11.784984,12.151931,1.971553e+10,3.297169e+10,3.627353e+09,5.768247e+09
93,1.354485e+10,2.403801e+09,1.125814e+09,538.496453,68.474851,1.364925e+09,5.817272e+07,0.213278,4.982616,11.238868,11.470734,2.926961e+10,4.217391e+10,5.314754e+09,7.367896e+09
94,3.054517e+10,5.330737e+09,3.912047e+09,613.466892,573.311932,4.018208e+09,1.227488e+08,0.250349,5.047734,11.525254,11.754408,5.803831e+09,1.354485e+10,1.125814e+09,2.403801e+09


## Drop คอลัมที่มีค่า Correlation มากกว่า 0.90

In [7]:
corr_matrix = feature_train.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

# Drop features 
feature_train.drop(to_drop, axis=1, inplace=True)
feature_test.drop(to_drop, axis=1, inplace=True)

## จัดการข้อมูลที่เป็นค่า 0 โดยใส่ค่า Mean

In [8]:
mean_p = feature_train['PRICE'].mean()
print(mean_p)
mean_pf = feature_train['PRICE_FACTORY'].mean()
print(mean_pf)

47.843799518531256
125.69256250418319


In [9]:
feature_train['PRICE'] = feature_train['PRICE'].replace([0],mean_p)
feature_train['PRICE_FACTORY'] = feature_train['PRICE_FACTORY'].replace([0],mean_pf)
feature_train

,TAX_AMT,VOLUMN,PRICE,PRICE_FACTORY,TOTAL_VOLUMN_CAPA,CAPA,VAL_RATE,TAXQTY_UNIT,ART_TAXV,LAG_TIME_TOTAL_TAX_AMT,LAG_TIME_TOTAL_VOLUMN
0,2.222302e+10,4.344679e+09,47.843800,125.692563,3.633069e+07,1.286312e+07,11.053448,5.076811,3.704450,0.000000e+00,0.000000e+00
1,2.164869e+10,4.220640e+09,47.843800,125.692563,7.478745e+07,3.279386e+06,11.229805,4.969179,5.021377,1.371691e+10,2.863381e+09
2,2.511702e+10,4.896327e+09,47.843800,125.692563,3.713671e+07,3.346770e+07,11.589345,5.101931,3.734156,1.407221e+10,2.914440e+09
3,2.261955e+10,4.597110e+09,47.843800,125.692563,3.694402e+07,3.134440e+07,11.864044,4.974018,3.488816,1.353646e+10,2.865928e+09
4,8.343328e+09,4.743274e+09,47.843800,125.692563,6.134105e+07,5.331400e+07,11.384126,3.887956,2.687062,1.324597e+10,2.848384e+09
...,...,...,...,...,...,...,...,...,...,...,...
91,3.297169e+10,5.768247e+09,459.402081,8.583704,3.774096e+09,9.677267e+07,0.234029,4.972036,10.560527,1.931377e+10,3.567040e+09
92,4.217391e+10,7.367896e+09,420.296695,59.373953,5.553845e+09,5.147209e+07,0.055230,5.021154,11.784984,1.971553e+10,3.627353e+09
93,1.354485e+10,2.403801e+09,538.496453,68.474851,1.364925e+09,5.817272e+07,0.213278,4.982616,11.238868,2.926961e+10,5.314754e+09
94,3.054517e+10,5.330737e+09,613.466892,573.311932,4.018208e+09,1.227488e+08,0.250349,5.047734,11.525254,5.803831e+09,1.125814e+09


## Model

In [10]:
regr = linear_model.LinearRegression()
regr.fit(feature_train,label_train)
pred = regr.predict(feature_test)

In [11]:
print(label_test)
print(pred)

96     2.184130e+10
97     2.230267e+10
98     2.150852e+10
99     2.282755e+10
100    2.283282e+10
101    2.235165e+10
102    2.248958e+10
103    2.093361e+10
104    2.113849e+10
105    2.907713e+10
106    1.639425e+10
107    1.697095e+10
Name: TOTAL_TAX_AMT, dtype: float64
[2.23015524e+10 2.07644159e+10 2.07022085e+10 2.12683291e+10
 2.20828420e+10 2.07081932e+10 2.05654707e+10 1.83059166e+10
 1.96042017e+10 3.31769866e+10 1.74147455e+10 1.84965765e+10]


In [12]:
test = df.iloc[96:,[0,2]]
test['ACTUAL'] = label_test
test['Predict'] = pred
test['MAE%'] = (test['ACTUAL']-test['Predict']).abs()/test['ACTUAL']*100
test

,CALENDAR_YEAR,MONTH_CD,ACTUAL,Predict,MAE%
96,2562,1,2.184130e+10,2.230155e+10,2.107269
97,2562,2,2.230267e+10,2.076442e+10,6.897157
98,2562,3,2.150852e+10,2.070221e+10,3.748822
99,2562,4,2.282755e+10,2.126833e+10,6.830430
100,2562,5,2.283282e+10,2.208284e+10,3.284642
101,2562,6,2.235165e+10,2.070819e+10,7.352715
102,2562,7,2.248958e+10,2.056547e+10,8.555573
103,2562,8,2.093361e+10,1.830592e+10,12.552510
104,2562,9,2.113849e+10,1.960420e+10,7.258283
105,2562,10,2.907713e+10,3.317699e+10,14.099939


# **Normalization**

## ทำการ normalization ข้อมูลทั้งหมด

In [13]:
df_ = df.iloc[:,4:]
df_

,TOTAL_TAX_AMT,VOLUMN,TOTAL_VOLUMN,PRICE,PRICE_FACTORY,TOTAL_VOLUMN_CAPA,CAPA,VAL_RATE,TAXQTY_UNIT,ART_TAXV,ART_TOTALTAXV,LAG_TIME_TOTAL_TAX_AMT,LAG_TIME_TAX_AMT,LAG_TIME_TOTAL_VOLUMN,LAG_TIME_VOLUMN,TOTAL_GASOLINE,KEROSENE,TOTAL_DIESEL,LPG,TOTAL_Fuel,GDP_Agriculture,GDP_Non_Agriculture,Total_GDP,Pop_up_15,Total_Labor_Froce,Labor_Agr,Labor_Non_Agr,P_ULG95,P_HSD,P_LPG,Crude_WPrice,Change_OP,CPI,Inf,Inter,Exr,U_Em,Total_Auto,Total_Auto_Petrol,Total_Auto_Diesel,Total_Auto_LPG,Total_Auto_CNG,Total_Auto_Electricity,Total_Auto_Hybrid,CPI_OA,W_Avr,W_Pri,W_Plu
0,1.371691e+10,4.344679e+09,2.863381e+09,0.000000,0.000000,3.633069e+07,1.286312e+07,11.053448,5.076811,3.704450,5.150043,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,633.632,1.172,1695.860,1105.3944,4113.0854,164351.67,1871498.67,2047414.0,54279.62,38619.17,13389.40,24537.52,25.030339,26.414387,10.43150,110.686000,2.815500,99.65,3.37,5.32,31.5755,0.81,30322761.0,21592247.0,7405637.0,843450.0,265920,6312.0,22767.0,106.330000,10547.01,8964.45,16943.19
1,1.407221e+10,4.220640e+09,2.914440e+09,0.000000,0.000000,7.478745e+07,3.279386e+06,11.229805,4.969179,5.021377,6.429500,1.371691e+10,2.222302e+10,2.863381e+09,4.344679e+09,605.778,1.391,1698.912,1013.9074,3948.4414,164701.33,1937753.33,2109519.0,54322.32,38798.15,13449.78,24607.89,26.664445,27.770914,10.91669,119.327000,8.641000,101.33,3.34,5.25,30.7303,0.66,30483912.0,21695114.0,7444115.0,856446.0,270762,6228.0,23945.0,108.550000,10579.26,8778.73,17839.65
2,1.353646e+10,4.896327e+09,2.865928e+09,0.000000,0.000000,3.713671e+07,3.346770e+07,11.589345,5.101931,3.734156,5.349773,1.407221e+10,2.164869e+10,2.914440e+09,4.220640e+09,639.865,1.342,1831.691,1152.6093,4269.9543,165051.00,2004008.00,2171624.0,54365.40,38955.14,13506.88,24688.20,27.047719,27.639726,10.27190,125.445455,6.118455,98.57,3.45,5.25,30.6901,0.73,30667049.0,21809442.0,7490151.0,872606.0,275828,6133.0,25009.0,110.260000,10817.51,9267.39,17432.88
3,1.324597e+10,4.597110e+09,2.848384e+09,0.000000,0.000000,3.694402e+07,3.134440e+07,11.864044,4.974018,3.488816,5.101580,1.353646e+10,2.511702e+10,2.865928e+09,4.896327e+09,614.701,1.244,1724.461,1123.4093,4106.2233,163452.00,2024071.00,2185274.0,54408.85,38909.27,13572.62,24571.16,27.184130,27.380147,10.25900,119.421667,-6.023788,98.49,2.47,5.24,30.8884,0.97,30804836.0,21887534.0,7531226.0,886244.0,279621,6059.0,25598.0,111.583333,11453.95,9750.22,18382.53
4,4.914525e+09,4.743274e+09,2.967009e+09,0.000000,0.000000,6.134105e+07,5.331400e+07,11.384126,3.887956,2.687062,3.752981,1.324597e+10,2.261955e+10,2.848384e+09,4.597110e+09,632.200,1.261,1806.773,1154.9130,4307.4840,161853.00,2044134.00,2198924.0,54452.70,39006.98,14393.90,23874.57,25.156665,25.870855,10.32540,110.339524,-9.082143,99.20,2.52,5.23,31.3373,0.92,31037272.0,22033426.0,7590982.0,905418.0,284986,5963.0,26932.0,109.716667,11137.12,9548.08,17200.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2.093361e+10,5.434510e+09,3.818035e+09,49.936558,67.880428,5.201406e+09,2.133556e+08,0.009600,5.100187,10.758439,11.020983,2.248958e+10,3.238666e+10,4.066556e+09,5.636718e+09,0.000,0.000,0.000,0.0000,0.0000,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00000,44.736000,1.493826,0.00,0.00,0.00,31.2150,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.00,0.00,0.00
104,2.113849e+10,5.380393e+09,3.795921e+09,471.270093,490.512497,4.631266e+09,1.352323e+08,0.064000,5.085954,7.666068,7.959199,2.093361e+10,3.106545e+10,3.818035e+09,5.434510e+09,0.000,0.000,0.000,0.0000,0.0000,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.00,0.00,0.00
105,2.907713e+10,9.029945e+09,5.490935e+09,398.927588,410.381595,7.118240e+09,4.039597e+08,0.050032,5.129802,5.415956,6.702060,2.113849e+10,3.081077e+10,3.795921e+09,5.380393e+09,0.000,0.000,0.000,0

In [14]:
df_nor = df_.copy() #df_normalization
for i in range(len(df_nor.columns)-1): #normalization
  df_nor.iloc[:,i] = (df_nor.iloc[:,i] - df_nor.iloc[:,i].min()) / (df_nor.iloc[:,i].max() - df_nor.iloc[:,i].min())
df_nor

,TOTAL_TAX_AMT,VOLUMN,TOTAL_VOLUMN,PRICE,PRICE_FACTORY,TOTAL_VOLUMN_CAPA,CAPA,VAL_RATE,TAXQTY_UNIT,ART_TAXV,ART_TOTALTAXV,LAG_TIME_TOTAL_TAX_AMT,LAG_TIME_TAX_AMT,LAG_TIME_TOTAL_VOLUMN,LAG_TIME_VOLUMN,TOTAL_GASOLINE,KEROSENE,TOTAL_DIESEL,LPG,TOTAL_Fuel,GDP_Agriculture,GDP_Non_Agriculture,Total_GDP,Pop_up_15,Total_Labor_Froce,Labor_Agr,Labor_Non_Agr,P_ULG95,P_HSD,P_LPG,Crude_WPrice,Change_OP,CPI,Inf,Inter,Exr,U_Em,Total_Auto,Total_Auto_Petrol,Total_Auto_Diesel,Total_Auto_LPG,Total_Auto_CNG,Total_Auto_Electricity,Total_Auto_Hybrid,CPI_OA,W_Avr,W_Pri,W_Plu
0,0.447422,0.364189,0.416840,0.000000,0.000000,0.000538,0.023458,0.889972,0.306422,0.002637,0.004087,0.000000,0.000000,0.000000,0.000000,0.589826,0.804393,0.774727,0.893609,0.808158,0.920212,0.714659,0.746756,0.956151,0.960869,0.793593,0.896604,0.920770,0.951153,0.467659,0.882344,0.763755,0.858460,0.964570,1.000000,0.873187,0.613636,0.737886,0.771129,0.644115,0.675409,0.637514,1.000000,0.148625,0.931113,0.721251,0.684920,16943.19
1,0.460046,0.347356,0.428172,0.000000,0.000000,0.001405,0.002933,0.904174,0.287901,0.005368,0.006747,0.468640,0.432706,0.521474,0.481141,0.563898,0.954701,0.776121,0.819650,0.775808,0.922170,0.739960,0.769408,0.956904,0.965322,0.797172,0.899176,0.980883,1.000000,0.489411,0.951226,0.931899,0.872932,0.960319,0.986842,0.849814,0.500000,0.741807,0.774803,0.647462,0.685816,0.649123,0.986692,0.156315,0.950553,0.723457,0.670730,17839.65
2,0.441011,0.439050,0.417405,0.000000,0.000000,0.000557,0.067587,0.933126,0.310745,0.002698,0.004503,0.480779,0.421523,0.530773,0.467405,0.595628,0.921071,0.836779,0.931778,0.838980,0.924127,0.765260,0.792059,0.957663,0.969228,0.800556,0.902110,0.994982,0.995276,0.460504,1.000000,0.859090,0.849156,0.975908,0.986842,0.848702,0.553030,0.746264,0.778886,0.651466,0.698757,0.661268,0.971641,0.163261,0.965527,0.739749,0.708066,17432.88
3,0.430690,0.398445,0.413512,0.000000,0.000000,0.000552,0.063040,0.955246,0.288734,0.002190,0.003987,0.462475,0.489055,0.521938,0.542232,0.572204,0.853809,0.787793,0.908172,0.806810,0.915175,0.772921,0.797038,0.958428,0.968087,0.804453,0.897833,1.000000,0.985929,0.459926,0.951981,0.508625,0.848467,0.837024,0.984962,0.854186,0.734848,0.749617,0.781675,0.655039,0.709678,0.670361,0.959918,0.167106,0.977116,0.783272,0.744956,18382.53
4,0.134679,0.418280,0.439839,0.000000,0.000000,0.001102,0.110092,0.916600,0.101845,0.000527,0.001183,0.452550,0.440427,0.518743,0.509096,0.588493,0.865477,0.825396,0.933640,0.846354,0.906222,0.780583,0.802017,0.959200,0.970518,0.853130,0.872380,0.925417,0.931581,0.462903,0.879582,0.420350,0.854583,0.844110,0.983083,0.866600,0.696970,0.755273,0.786885,0.660236,0.725032,0.683223,0.944708,0.175815,0.960769,0.761606,0.729512,17200.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.703827,0.512084,0.628716,0.056613,0.021015,0.116879,0.452855,0.000654,0.310445,0.017264,0.016291,0.768360,0.630602,0.740594,0.624225,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.356617,0.725608,0.000000,0.486980,0.000000,0.863218,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
104,0.711107,0.504740,0.623808,0.534278,0.151857,0.104037,0.285538,0.005035,0.307996,0.010852,0.009927,0.715200,0.604877,0.695334,0.601832,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.682491,0.000000,0.486980,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
105,0.993161,1.000000,1.000000,0.452264,0.127049,0.160054,0.861074,0.003910,0.315541,0.006186,0.007313,0.722199,0.599918,0.691307,0.595839,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

## แบ่งข้อมูลTrain 2554-2561 Test 2562

In [15]:
feature_train_nor = df_nor.iloc[:96,1:15]
label_train_nor = df_nor.iloc[:96,0]
feature_test_nor = df_nor.iloc[96:,1:15]
label_test_nor = df_nor.iloc[96:,0]
feature_train_nor

,VOLUMN,TOTAL_VOLUMN,PRICE,PRICE_FACTORY,TOTAL_VOLUMN_CAPA,CAPA,VAL_RATE,TAXQTY_UNIT,ART_TAXV,ART_TOTALTAXV,LAG_TIME_TOTAL_TAX_AMT,LAG_TIME_TAX_AMT,LAG_TIME_TOTAL_VOLUMN,LAG_TIME_VOLUMN
0,0.364189,0.416840,0.000000,0.000000,0.000538,0.023458,0.889972,0.306422,0.002637,0.004087,0.000000,0.000000,0.000000,0.000000
1,0.347356,0.428172,0.000000,0.000000,0.001405,0.002933,0.904174,0.287901,0.005368,0.006747,0.468640,0.432706,0.521474,0.481141
2,0.439050,0.417405,0.000000,0.000000,0.000557,0.067587,0.933126,0.310745,0.002698,0.004503,0.480779,0.421523,0.530773,0.467405
3,0.398445,0.413512,0.000000,0.000000,0.000552,0.063040,0.955246,0.288734,0.002190,0.003987,0.462475,0.489055,0.521938,0.542232
4,0.418280,0.439839,0.000000,0.000000,0.001102,0.110092,0.916600,0.101845,0.000527,0.001183,0.452550,0.440427,0.518743,0.509096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0.557373,0.586396,0.520823,0.002657,0.084729,0.203168,0.018726,0.288393,0.016854,0.015966,0.659857,0.616867,0.649623,0.623557
92,0.774453,0.960898,0.476490,0.018381,0.124817,0.106148,0.004328,0.296845,0.019393,0.018642,0.673584,0.641994,0.660607,0.638791
93,0.100803,0.031204,0.610493,0.021199,0.030464,0.120498,0.017055,0.290213,0.018261,0.017226,1.000000,0.821171,0.967914,0.815940
94,0.498001,0.649581,0.695486,0.177490,0.090228,0.258802,0.020041,0.301419,0.018854,0.017815,0.198289,0.263733,0.205031,0.266203


## Drop คอลัมที่มีค่า Correlation มากกว่า 0.90

In [16]:
corr_matrix = feature_train_nor.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

# Drop features 
feature_train_nor.drop(to_drop, axis=1, inplace=True)
feature_test_nor.drop(to_drop, axis=1, inplace=True)

## จัดการข้อมูลที่เป็นค่า 0 โดยใส่ค่า Mean

In [17]:
mean_p_nor = feature_train_nor['PRICE'].mean()
print(mean_p_nor)
mean_pf_nor = feature_train_nor['PRICE_FACTORY'].mean()
print(mean_pf_nor)

0.054240438300075855
0.03891287311553241


In [18]:
feature_train_nor['PRICE'] = feature_train_nor['PRICE'].replace([0],mean_p_nor)
feature_train_nor['PRICE_FACTORY'] = feature_train_nor['PRICE_FACTORY'].replace([0],mean_pf_nor)
feature_train_nor

,VOLUMN,PRICE,PRICE_FACTORY,TOTAL_VOLUMN_CAPA,CAPA,VAL_RATE,TAXQTY_UNIT,ART_TAXV,LAG_TIME_TOTAL_TAX_AMT,LAG_TIME_TOTAL_VOLUMN
0,0.364189,0.054240,0.038913,0.000538,0.023458,0.889972,0.306422,0.002637,0.000000,0.000000
1,0.347356,0.054240,0.038913,0.001405,0.002933,0.904174,0.287901,0.005368,0.468640,0.521474
2,0.439050,0.054240,0.038913,0.000557,0.067587,0.933126,0.310745,0.002698,0.480779,0.530773
3,0.398445,0.054240,0.038913,0.000552,0.063040,0.955246,0.288734,0.002190,0.462475,0.521938
4,0.418280,0.054240,0.038913,0.001102,0.110092,0.916600,0.101845,0.000527,0.452550,0.518743
...,...,...,...,...,...,...,...,...,...,...
91,0.557373,0.520823,0.002657,0.084729,0.203168,0.018726,0.288393,0.016854,0.659857,0.649623
92,0.774453,0.476490,0.018381,0.124817,0.106148,0.004328,0.296845,0.019393,0.673584,0.660607
93,0.100803,0.610493,0.021199,0.030464,0.120498,0.017055,0.290213,0.018261,1.000000,0.967914
94,0.498001,0.695486,0.177490,0.090228,0.258802,0.020041,0.301419,0.018854,0.198289,0.205031


## Model

In [19]:
regr_nor = linear_model.LinearRegression()
regr_nor.fit(feature_train_nor,label_train_nor)
pred_nor = regr_nor.predict(feature_test_nor)

In [20]:
print(label_test_nor)
print(pred_nor)

96     0.736077
97     0.752469
98     0.724254
99     0.771118
100    0.771305
101    0.754209
102    0.759110
103    0.703827
104    0.711107
105    0.993161
106    0.542546
107    0.563036
Name: TOTAL_TAX_AMT, dtype: float64
[0.73721514 0.69476578 0.71937967 0.71639632 0.76118347 0.71546259
 0.703508   0.61933099 0.65287727 1.13526496 0.59247505 0.59777292]


In [21]:
test_nor = df.iloc[96:,[0,2]]
test_nor['ACTUAL'] = label_test_nor
test_nor['Predict'] = pred_nor
test_nor['MAE%'] = (test_nor['ACTUAL']-test_nor['Predict']).abs()/test_nor['ACTUAL']*100
test_nor

,CALENDAR_YEAR,MONTH_CD,ACTUAL,Predict,MAE%
96,2562,1,0.736077,0.737215,0.154662
97,2562,2,0.752469,0.694766,7.668498
98,2562,3,0.724254,0.719380,0.672946
99,2562,4,0.771118,0.716396,7.096360
100,2562,5,0.771305,0.761183,1.312238
101,2562,6,0.754209,0.715463,5.137366
102,2562,7,0.759110,0.703508,7.324619
103,2562,8,0.703827,0.619331,12.005242
104,2562,9,0.711107,0.652877,8.188540
105,2562,10,0.993161,1.135265,14.308215


In [30]:
print('intercept',regr_nor.intercept_)


intercept 0.09910156964953709


In [32]:
print('coefficient',regr_nor.coef_)


coefficient [ 0.92265515  0.09954554  0.03477391  0.02266729  0.04106307 -0.08074786
  0.09858102 -0.01929255  0.76735397 -0.8097284 ]


In [38]:
print('adjusted r square',r2_score(label_test_nor,pred_nor))

adjusted r square 0.6721293522973857


#เป็นการทำนายปี2563

In [45]:
x0 = feature_train_nor.iloc[:,0].mean()
x1 = feature_train_nor.iloc[:,1].mean()
x2 = feature_train_nor.iloc[:,2].mean()
x3 = feature_train_nor.iloc[:,3].mean()
x4 = feature_train_nor.iloc[:,4].mean()
x5 = feature_train_nor.iloc[:,5].mean()
x6 = feature_train_nor.iloc[:,6].mean()
x7 = feature_train_nor.iloc[:,7].mean()
x8 = feature_train_nor.iloc[:,8].mean()
x9 = feature_train_nor.iloc[:,9].mean()



In [49]:
pred_2563 = regr_nor.predict([[x0,x1,x2,x3,x4,x5,x6,x7,x8,x9]])
print('predict 2563 =',pred_2563)

predict 2563 = [0.35673017]


# **สมาชิก**
นายพรทวี ทวีสินธุ์ รหัส 61050246<br>
นายฟาริซ หะยีมาสาและ รหัส 61050258